# CSC380 Decision Tree Classifier Project

<div>
        Members: Sebastian Arana and _
</div>

In [1]:
import warnings
# Suppress warnings
warnings.filterwarnings("ignore")

# Imports
<div>We are using scikit-learn, numpy, pandas, matplotlib</div>

In [3]:
from sklearn import datasets

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,ConfusionMatrixDisplay